In [ ]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.8.*"

     |████████████████████████████████| 4.9 MB 28.1 MB/s 
     |████████████████████████████████| 498.0 MB 13 kB/s 
     |████████████████████████████████| 5.8 MB 69.4 MB/s 
     |████████████████████████████████| 462 kB 74.1 MB/s 
     |████████████████████████████████| 1.4 MB 58.2 MB/s 


In [ ]:
!pip install -q tf-models-official==2.7.0

     |████████████████████████████████| 1.8 MB 41.9 MB/s 
     |████████████████████████████████| 352 kB 75.3 MB/s 
     |████████████████████████████████| 1.1 MB 61.5 MB/s 
     |████████████████████████████████| 1.3 MB 71.9 MB/s 
     |████████████████████████████████| 238 kB 84.3 MB/s 
     |████████████████████████████████| 43 kB 2.2 MB/s 
     |████████████████████████████████| 118 kB 65.1 MB/s 


In [ ]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

from keras import backend as K 

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [ ]:
import pandas as pd

train_all = pd.read_csv('texts_with_skills_one-hot.csv', sep = "|", encoding = "UTF-8")

In [ ]:
#train_all.head()

In [ ]:
train_all_X = train_all['text']

In [ ]:
#bert_preprocess_model = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_preprocess/1')

In [ ]:
#text_preprocessed = bert_preprocess_model(train_all_X)

In [ ]:
#bert_model = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_L-12_H-768_A-12/1')

In [ ]:
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_preprocess/1', name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer('https://tfhub.dev/jeongukjae/xlm_roberta_multi_cased_L-12_H-768_A-12/1', trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [ ]:
#classifier_model = build_classifier_model()

In [ ]:
#loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
#metrics = tf.metrics.BinaryAccuracy()

In [ ]:
#epochs = 10
#steps_per_epoch = 64
#num_train_steps = steps_per_epoch * epochs
#num_warmup_steps = int(0.1*num_train_steps)

#init_lr = 3e-5
#optimizer = optimization.create_optimizer(init_lr=init_lr,
#                                          num_train_steps=num_train_steps,
#                                          num_warmup_steps=num_warmup_steps,
#                                          optimizer_type='adamw')

In [ ]:
#classifier_model.compile(optimizer=optimizer,
#                         loss=loss,
#                         metrics=metrics)

In [ ]:
#history = classifier_model.fit(x = train_all_X, y = train_0_y, epochs = epochs)

Test

In [ ]:
test_all = pd.read_csv('test_texts_with_ids.csv', sep = "|", encoding = "UTF-8")

In [ ]:
test_texts = test_all["text"]

In [ ]:
#for i in tf.sigmoid(classifier_model(tf.constant(test_texts))):
#    print(i)

In [ ]:
#to_drop = ['s0', 's1', 's10', 's11', 's12', 's13', 's15', 's16', 's17', 's18', 's19', 's2']
skill_list = train_all.keys().drop('text').drop(to_drop).tolist()

In [ ]:
skills_in_test = []

for skill in skill_list:
    K.clear_session()
    tf.keras.backend.clear_session()
    tf.reset_default_graph()
    
    print("Class =", skill)
    classifier_model = build_classifier_model()
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()

    epochs = 10
    steps_per_epoch = 64
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)

    init_lr = 3e-5
    optimizer = optimization.create_optimizer(init_lr=init_lr,
                                            num_train_steps=num_train_steps,
                                            num_warmup_steps=num_warmup_steps,
                                            optimizer_type='adamw')
    
    classifier_model.compile(optimizer=optimizer,
                            loss=loss,
                            metrics=metrics)
    
    train_y = train_all[skill]

    total = len(train_y)
    pos = train_y.value_counts()[1]
    neg = train_y.value_counts()[0]

    weight_for_0 = (1 / neg) * (total / 2.0)
    weight_for_1 = (1 / pos) * (total / 2.0)

    class_weight = {0: weight_for_0, 1: weight_for_1}

    history = classifier_model.fit(x = train_all_X, y = train_y, epochs = epochs, class_weight = class_weight)

    filename = "test_preds_" + skill + ".txt"
    with open(filename, 'w') as fp:
        for item in tf.sigmoid(classifier_model(tf.constant(test_texts))).numpy().tolist():
            fp.write("%s\n" % item[0])
        print('Done')

    del classifier_model
    K.clear_session()

Class = s18
Epoch 1/10
7/7 [==============================] - 23s 899ms/step - loss: 0.7031 - binary_accuracy: 0.8079
Epoch 2/10
7/7 [==============================] - 6s 910ms/step - loss: 0.7055 - binary_accuracy: 0.9015
Epoch 3/10
7/7 [==============================] - 6s 881ms/step - loss: 0.7075 - binary_accuracy: 0.9212
Epoch 4/10
7/7 [==============================] - 6s 883ms/step - loss: 0.6887 - binary_accuracy: 0.9212
Epoch 5/10
7/7 [==============================] - 6s 893ms/step - loss: 0.6647 - binary_accuracy: 0.9507
Epoch 6/10
7/7 [==============================] - 7s 1s/step - loss: 0.6263 - binary_accuracy: 0.9507
Epoch 7/10
7/7 [==============================] - 6s 900ms/step - loss: 0.7764 - binary_accuracy: 0.9507
Epoch 8/10
7/7 [==============================] - 7s 976ms/step - loss: 0.7711 - binary_accuracy: 0.9507
Epoch 9/10
7/7 [==============================] - 7s 918ms/step - loss: 0.6469 - binary_accuracy: 0.9507
Epoch 10/10
7/7 [============================

ResourceExhaustedError: ignored

In [62]:
import os
from natsort import natsorted
# assign directory
directory = 'preds'

skills_preds = pd.DataFrame()

# iterate over files in
# that directory
for filename in natsorted(os.listdir(directory)):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        skill = f.strip().split(".")[0].split("_")[-1]
        with open(f) as file:
          df = file.readlines()
          skills_preds[skill] = [float(i) for i in df]

In [78]:
truths = pd.concat([test_all.drop('text', axis = 1), skills_preds >= 0.5], axis = 1)

In [126]:
thingie = []

for index, row in truths.iterrows():
  id = row['text_id']

  skl = row.drop('text_id')

  true_skills = skl[skl == True]
  
  if len(true_skills) == 0:
    highest_idx = skills_preds.iloc[index].idxmax()
    true_skills = [highest_idx]
  else:
    true_skills = true_skills.keys().to_list()

  skill_list = " ".join(true_skills)
  thingie.append(",".join([id, skill_list]))

In [127]:
with open("XLM_RoBERTa_10e_withBest.csv", "w") as file:
  file.write("text_id,skills\n")
  for line in thingie:
    file.write(line)
    file.write("\n")